# Tier 3: End-to-End Pricing

**Learning Objectives:**
- Price a complete annuity product
- Use the full annuity-pricing API
- Validate results against known values

**This notebook ties together:**
- Market parameters
- Product specifications
- Pricing engines
- Validation gates

**Duration:** ~25 minutes

In [ ]:
import numpy as np
from annuity_pricing.products import ProductRegistry, create_default_registry
from annuity_pricing.data.schemas import FIAProduct, RILAProduct, MYGAProduct
from annuity_pricing.products.fia import FIAPricer, MarketParams

## 1. Set Up Market Environment

In [ ]:
# Market parameters
market = MarketParams(
    spot=100.0,
    risk_free_rate=0.05,
    dividend_yield=0.02,
    volatility=0.20,
)

print("Market Environment:")
print(f"  Spot (S₀):      ${market.spot:.2f}")
print(f"  Risk-free (r):  {market.risk_free_rate:.2%}")
print(f"  Dividend (q):   {market.dividend_yield:.2%}")
print(f"  Volatility (σ): {market.volatility:.2%}")

## 2. Define Products

In [ ]:
# FIA with cap
fia_product = FIAProduct(
    company_name="Example Life",
    product_name="S&P 500 FIA",
    product_group="FIA",
    status="current",
    index_used="S&P 500",
    indexing_method="Annual Point to Point",
    cap_rate=0.10,  # 10% cap
)

print("FIA Product:")
print(f"  Product:   {fia_product.product_name}")
print(f"  Index:     {fia_product.index_used}")
print(f"  Method:    {fia_product.indexing_method}")
print(f"  Cap Rate:  {fia_product.cap_rate:.1%}")

## 3. Price the Product

In [ ]:
# Create pricer
pricer = FIAPricer(market_params=market)

# Price FIA
term_years = 7  # 7-year term
premium = 100_000  # $100k premium

result = pricer.price(fia_product, term_years=term_years, premium=premium)

print("\nPricing Results:")
print(f"  Term:           {term_years} years")
print(f"  Premium:        ${premium:,.0f}")
print(f"  Expected Credit: {result.expected_credit:.2%}")
print(f"  Option Cost:    ${result.option_cost:,.2f}")
print(f"  Present Value:  ${result.present_value:,.2f}")

## 4. Validate Results

In [ ]:
# Sanity checks
print("Validation Checks:")

# Expected credit should be positive and less than cap
credit_ok = 0 <= result.expected_credit <= fia_product.cap_rate
print(f"  Expected credit in [0, cap]: {'✓' if credit_ok else '✗'}")

# Option cost should be positive
option_ok = result.option_cost > 0
print(f"  Option cost > 0: {'✓' if option_ok else '✗'}")

# Present value should be positive
pv_ok = result.present_value > 0
print(f"  Present value > 0: {'✓' if pv_ok else '✗'}")

## 5. Complete Workflow Summary

1. **Set up market**: Define spot, rates, volatility
2. **Define product**: Create product dataclass
3. **Create pricer**: Initialize with market params
4. **Price**: Call `pricer.price(product, ...)`
5. **Validate**: Check sanity constraints

## Next Steps

- Compare with competitor products (WINK data)
- Sensitivity analysis on market parameters
- Cross-validate with external libraries